## Statoil Iceberg Classification

### Ensemble learning
### Stacking several CNN prediction result for better accuracy

In [3]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

Iceberg_py3_keras.ipynb
Iceberg_py3_keras_angle.ipynb
README.md
ensemble.ipynb
sub.csv
submission.csv
test.json
train.json



In [ ]:
sub_path = "./submissions"
all_files = os.listdir(sub_path)

outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()